In [37]:
import pandas as pd
import numpy as np
import string 

import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch import nn, optim

In [38]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device

In [39]:
device = torch.device('cpu')
device

device(type='cpu')

In [40]:
train_data = pd.read_json('DATA/MT_JAVNRF_INZNTV/train_preprocess.json')
test_data = pd.read_json('DATA/MT_JAVNRF_INZNTV/test_preprocess.json')

In [41]:
train_data.head()

,id,text,label
0,MRK_1_27,"Wong kabèh padha gumun, temah padha takon-tin...","Mereka semua takjub, sehingga mereka memperbin..."
1,REV_19_15,"Saka ing tutuké metu pedhang kang landhep, ka...",Dan dari mulut-Nya keluarlah sebilah pedang ta...
2,MRK_13_23,Déné kowé padha diawas! Aku wus ngandhakake...,Hati-hatilah kamu! Aku sudah terlebih dahulu m...
3,LUK_2_5,"supaya didhaftaraké bebarengan karo Maria, pa...","supaya didaftarkan bersama-sama dengan Maria, ..."
4,ACT_13_34,Allah sampun mungokaken Panjenenganipun saking...,Allah telah membangkitkan Dia dari antara oran...


In [42]:
test_data.head()

,id,text,label
0,MAT_14_1,"Nalika samana Hérodès, raja-wilayah, midhang...",Pada masa itu sampailah berita-berita tentang ...
1,ACT_26_12,"""Lan ing salebeting kawontenan ingkang makaten...","""Dan dalam keadaan demikian, ketika aku dengan..."
2,ACT_2_31,Margi saking punika panjenenganipun uninga sad...,Karena itu ia telah melihat ke depan dan telah...
3,HEB_13_25,Sih-rahmat nunggila ing kowé kabèh.,Kasih karunia menyertai kamu sekalian.
4,1CO_4_12,nganggo tanganku dhéwé aku nindakaké pagawe...,kami melakukan pekerjaan tangan yang berat. Ka...


In [43]:
train_data.drop(columns='id', inplace=True)
test_data.drop(columns='id', inplace=True)

In [44]:
train_data.head()

,text,label
0,"Wong kabèh padha gumun, temah padha takon-tin...","Mereka semua takjub, sehingga mereka memperbin..."
1,"Saka ing tutuké metu pedhang kang landhep, ka...",Dan dari mulut-Nya keluarlah sebilah pedang ta...
2,Déné kowé padha diawas! Aku wus ngandhakake...,Hati-hatilah kamu! Aku sudah terlebih dahulu m...
3,"supaya didhaftaraké bebarengan karo Maria, pa...","supaya didaftarkan bersama-sama dengan Maria, ..."
4,Allah sampun mungokaken Panjenenganipun saking...,Allah telah membangkitkan Dia dari antara oran...


DATA PREPROCESS 
- lower and remove quotes
- remove special char
- remove punctuation

In [45]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [46]:

import nltk
nltk.download("punkt")
import re

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from string import punctuation

[nltk_data] Downloading package punkt to C:\Users\Kania Galih
[nltk_data]     Widowati\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Resoruce for javanese stopwords 
- https://github.com/bimarakajati/Javanese-and-Sundanese-Stopwords.git 

In [47]:
javanese_stopwords = pd.read_csv("local_languages_stopwords.csv")
javanese_stopwords.head()

,indonesian,javanese,sundanese
0,ada,Ana,Aya
1,adalah,yaiku,geus
2,adanya,orane,ayana
3,adapun,Kangge,Samentara éta
4,agak,Luwih,rada


In [48]:
#remove sundanese column
javanese_stopwords.drop(columns=["indonesian", "sundanese"], inplace=True)
javanese_stopwords.head()

,javanese
0,Ana
1,yaiku
2,orane
3,Kangge
4,Luwih


In [49]:
javanese_stopwords = javanese_stopwords['javanese'].tolist()
javanese_stopwords

['Ana',
 'yaiku',
 'orane',
 'Kangge',
 'Luwih',
 'Dianggo',
 'dadi ngono',
 'bakal',
 'bakal',
 'pungkasan',
 'pungkasan',
 'Pungkasane',
 'Aku',
 'kula',
 'banget',
 'Banget',
 'Sampeyan',
 'Sampeyan',
 'antarane',
 'antarane',
 'Antarane wong-wong mau',
 'Apa',
 'Apa',
 'Yen',
 'yaiku',
 'utamane',
 'Akatah',
 'Tegese',
 'Asal',
 'diwenehake',
 'ing',
 'utawa',
 'utawa iku',
 'utawa',
 'diwiwiti',
 'Wiwitane',
 'kaya',
 'kaya',
 'Kepiye',
 'Kepiye',
 'Nanging',
 'kanggo',
 'bagean',
 'Malah',
 'iku',
 'iku',
 'Apik',
 'bakal',
 'arep',
 'teka maneh',
 'Akeh',
 'Rama',
 'Anyar',
 'ngisor',
 'nomer saka',
 'Katon',
 'iki',
 'Kajaba iku',
 'Iki carane',
 'dadi',
 'Kajaba iku',
 'Iku iku',
 'Kajaba iku',
 'Nyambut gawe',
 'konco',
 'Mengko',
 'Durung',
 'durung',
 'Bener',
 'Iku bener',
 'Bener',
 'ana ing',
 'pungkasan',
 'pungkasan',
 'rampung',
 'Wonten pinten',
 'Apa',
 'Apa sing sampeyan lakoni',
 'apa wae',
 'Tegese',
 'Miwiti',
 'Macem-macem',
 'Tiba',
 'Menehi',
 'menehi',
 'Dip

In [50]:
nltk.download('stopwords', quiet=True)
indonesian_stopwords = stopwords.words('indonesian')

In [51]:
indonesian_stopwords

['ada',
 'adalah',
 'adanya',
 'adapun',
 'agak',
 'agaknya',
 'agar',
 'akan',
 'akankah',
 'akhir',
 'akhiri',
 'akhirnya',
 'aku',
 'akulah',
 'amat',
 'amatlah',
 'anda',
 'andalah',
 'antar',
 'antara',
 'antaranya',
 'apa',
 'apaan',
 'apabila',
 'apakah',
 'apalagi',
 'apatah',
 'artinya',
 'asal',
 'asalkan',
 'atas',
 'atau',
 'ataukah',
 'ataupun',
 'awal',
 'awalnya',
 'bagai',
 'bagaikan',
 'bagaimana',
 'bagaimanakah',
 'bagaimanapun',
 'bagi',
 'bagian',
 'bahkan',
 'bahwa',
 'bahwasanya',
 'baik',
 'bakal',
 'bakalan',
 'balik',
 'banyak',
 'bapak',
 'baru',
 'bawah',
 'beberapa',
 'begini',
 'beginian',
 'beginikah',
 'beginilah',
 'begitu',
 'begitukah',
 'begitulah',
 'begitupun',
 'bekerja',
 'belakang',
 'belakangan',
 'belum',
 'belumlah',
 'benar',
 'benarkah',
 'benarlah',
 'berada',
 'berakhir',
 'berakhirlah',
 'berakhirnya',
 'berapa',
 'berapakah',
 'berapalah',
 'berapapun',
 'berarti',
 'berawal',
 'berbagai',
 'berdatangan',
 'beri',
 'berikan',
 'berikut'

In [52]:
def preprocess_text(text, language_label):
    #lowercase
    text = text.lower()
    #remove quotes
    text = text.strip('"')
    # Combine removing special characters, punctuation, and extra whitespace
    text = re.sub(r"[^\w\s]", "", text)  # Removes special characters
    text = "".join([char for char in text if char not in punctuation])  # Removes punctuation
    text = " ".join(text.split())  # Removes extra whitespace
    #Tokenize 
    tokens = word_tokenize(text)

    # Select stopwords based on language label
    if language_label == "label":
        stopwords = indonesian_stopwords
    elif language_label == "text":
        stopwords = javanese_stopwords
    else:
        # Default to empty set if language label is not recognized
        stopwords = set()

    # Remove stopwords
    filtered_tokens = [word for word in tokens if word not in stopwords]
    

    # Join the tokens back into a string
    preprocessed_text = ' '.join(filtered_tokens)
    
    return preprocessed_text


In [53]:
train_data['label'] = train_data['label'].apply(lambda x: preprocess_text(x, indonesian_stopwords))
train_data['text'] = train_data['text'].apply(lambda x: preprocess_text(x, javanese_stopwords))

In [54]:
test_data['label'] = test_data['label'].apply(lambda x: preprocess_text(x, indonesian_stopwords))
test_data['text'] = test_data['text'].apply(lambda x: preprocess_text(x, javanese_stopwords))

In [55]:
train_data.head()

,text,label
0,wong kabeh padha gumun temah padha takontinako...,mereka semua takjub sehingga mereka memperbinc...
1,saka ing tutuke metu pedhang kang landhep kang...,dan dari mulutnya keluarlah sebilah pedang taj...
2,dene kowe padha diawas aku wus ngandhakake iki...,hatihatilah kamu aku sudah terlebih dahulu men...
3,supaya didhaftarake bebarengan karo maria paca...,supaya didaftarkan bersamasama dengan maria tu...
4,allah sampun mungokaken panjenenganipun saking...,allah telah membangkitkan dia dari antara oran...


In [56]:
test_data.head()

,text,label
0,nalika samana herodes rajawilayah midhanget pa...,pada masa itu sampailah beritaberita tentang y...
1,lan ing salebeting kawontenan ingkang makaten ...,dan dalam keadaan demikian ketika aku dengan k...
2,margi saking punika panjenenganipun uninga sad...,karena itu ia telah melihat ke depan dan telah...
3,sihrahmat nunggila ing kowe kabeh,kasih karunia menyertai kamu sekalian
4,nganggo tanganku dhewe aku nindakake pagawean ...,kami melakukan pekerjaan tangan yang berat kal...


In [57]:
len(train_data)

5968

In [58]:
len(test_data)

1193

Change to tensor 

In [59]:
src_data_train = train_data.text
trg_data_train = train_data.label

src_data_test = test_data.text
trg_data_test = test_data.label


In [60]:
from collections import defaultdict

def preprocess_data(src_data, trg_data):
    # Tokenize sentences using a suitable tokenizer
    src_tokens = [source.split() for source in src_data]
    trg_tokens = [target.split() for target in trg_data]

    # Create vocabularies with explicit handling of unknown words
    src_vocab = defaultdict(lambda: len(src_vocab), {"<unk>": 0})
    trg_vocab = defaultdict(lambda: len(trg_vocab), {"<unk>": 0})
    
    # Update vocabularies with tokens
    for src_sent, trg_sent in zip(src_tokens, trg_tokens):
        for word in src_sent:
            src_vocab[word]
        for word in trg_sent:
            trg_vocab[word]
    
    # Convert tokens to index
    src_indices = [[src_vocab[word] for word in sent] for sent in src_tokens]
    trg_indices = [[trg_vocab[word] for word in sent] for sent in trg_tokens]
    
    # Pad sequences to the maximum length
    max_length = max(max(len(sent) for sent in src_indices), max(len(sent) for sent in trg_indices))
    src_indices = [sent + [0] * (max_length - len(sent)) for sent in src_indices]
    trg_indices = [sent + [0] * (max_length - len(sent)) for sent in trg_indices]
    
    # Convert indices to tensors
    src_tensors = torch.tensor(src_indices)
    trg_tensors = torch.tensor(trg_indices)
    
    return src_tensors, trg_tensors, src_vocab, trg_vocab

In [61]:
src_tensors_train, trg_tensors_train, src_vocab_train, trg_vocab_train = preprocess_data(src_data_train, trg_data_train)
print("Source tensors:", src_tensors_train)
print("Target tensors:", trg_tensors_train)
print("Source vocabulary:", src_vocab_train)
print("Target vocabulary:", trg_vocab_train)

Source tensors: tensor([[  1,   2,   3,  ...,   0,   0,   0],
        [ 27,  28,  29,  ...,   0,   0,   0],
        [ 50,  51,   3,  ...,   0,   0,   0],
        ...,
        [117, 118,   1,  ...,   0,   0,   0],
        [ 53,   2,  24,  ...,   0,   0,   0],
        [209, 361,  51,  ...,   0,   0,   0]])
Target tensors: tensor([[   1,    2,    3,  ...,    0,    0,    0],
        [  20,   23,   24,  ...,    0,    0,    0],
        [  46,   47,   48,  ...,    0,    0,    0],
        ...,
        [ 104,  372,   12,  ...,    0,    0,    0],
        [ 562, 2448,  116,  ...,    0,    0,    0],
        [ 140,   73,  211,  ...,    0,    0,    0]])
Source vocabulary: defaultdict(<function preprocess_data.<locals>.<lambda> at 0x000001959394C040>, {'<unk>': 0, 'wong': 1, 'kabeh': 2, 'padha': 3, 'gumun': 4, 'temah': 5, 'takontinakon': 6, 'kalawan': 7, 'ngucap': 8, 'apa': 9, 'ta': 10, 'iki': 11, 'piwulang': 12, 'anyar': 13, 'mawa': 14, 'pangwasa': 15, 'apadene': 16, 'paring': 17, 'dhawuh': 18, 'mar

In [62]:
src_tensors_test, trg_tensors_test, src_vocab_test, trg_vocab_test = preprocess_data(src_data_test, trg_data_test)
print("Source tensors:", src_tensors_test)
print("Target tensors:", trg_tensors_test)
print("Source vocabulary:", src_vocab_test)
print("Target vocabulary:", trg_vocab_test)

Source tensors: tensor([[   1,    2,    3,  ...,    0,    0,    0],
        [   9,   10,   11,  ...,    0,    0,    0],
        [  29,   30,   15,  ...,    0,    0,    0],
        ...,
        [ 360,  180, 4143,  ...,    0,    0,    0],
        [1097,   73,  994,  ...,    0,    0,    0],
        [1050,   85,  209,  ...,    0,    0,    0]])
Target tensors: tensor([[  1,   2,   3,  ...,   0,   0,   0],
        [ 12,  13,  14,  ...,   0,   0,   0],
        [ 29,   3,  30,  ...,   0,   0,   0],
        ...,
        [133, 214,  29,  ...,   0,   0,   0],
        [526, 308,  77,  ...,   0,   0,   0],
        [305,  67, 239,  ...,   0,   0,   0]])
Source vocabulary: defaultdict(<function preprocess_data.<locals>.<lambda> at 0x000001959394C4A0>, {'<unk>': 0, 'nalika': 1, 'samana': 2, 'herodes': 3, 'rajawilayah': 4, 'midhanget': 5, 'pawarta': 6, 'bab': 7, 'yesus': 8, 'lan': 9, 'ing': 10, 'salebeting': 11, 'kawontenan': 12, 'ingkang': 13, 'makaten': 14, 'punika': 15, 'kula': 16, 'kanthi': 17, 'ke

In [63]:
#create dataset & dataloader 
train_set = TensorDataset(src_tensors_train, trg_tensors_train)
trainloader = DataLoader(train_set, batch_size=32)

test_set = TensorDataset(src_tensors_test, trg_tensors_test)
testloader = DataLoader(train_set, batch_size=32)

In [64]:
class Seq2Seq(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super(Seq2Seq, self).__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim) #change word to vector numeric  
        self.encoder = nn.LSTM(embedding_dim, hidden_dim) #extract fitur 
        self.decoder = nn.LSTM(hidden_dim, hidden_dim) #produces a new output sequence 
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, src, trg):
        embedded_src = self.embedding(src)
        embedded_trg = self.embedding(trg)  
        encoder_outputs, (hidden, cell) = self.encoder(embedded_src)
        decoder_outputs, _ = self.decoder(embedded_trg, (hidden, cell))  
        output = self.fc_out(decoder_outputs)
        return output

In [65]:
input_dim = 20  # Size of the input vocabulary
output_dim = 20  # Size of the output vocabulary
embedding_dim = 32
hidden_dim = 256

In [66]:
model = Seq2Seq(input_dim, embedding_dim, hidden_dim,output_dim).to(device)
optimizer = optim.AdamW(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

TRAIN

In [67]:
from tqdm.auto import tqdm

def loop_fn(mode, dataset, dataloader, model, criterion, optimizer, device):
    #Mode checking
    if mode == "train":
        model.train()
    elif mode == "test":
        model.eval()

    #define cost
    cost = 0

    #TRAINING LOOP -> backpropagation only in training phase
    for feature, target in tqdm(dataloader, desc=mode.title()): #to display its train or test
        if feature is None or target is None:
            continue
        #change the minibatch to gpu
        feature, target = feature.to(device), target.to(device)
        #feedforward
        output= model(feature, None)
        #loss
        loss = criterion(output, target)

        if mode == "train":
            #backpropagation
            loss.backward()
            #update weight
            optimizer.step()
            #reset optimizer
            optimizer.zero_grad()

        #calculate total loss
        cost += loss.item() * feature.shape[0] 

    #calculate avarge loss
    cost = cost / len(dataset)
    return cost

In [68]:
while True:
    #train
    train_cost = loop_fn("train", train_set, trainloader, model, criterion, optimizer, device)

    #test
    with torch.no_grad():
        test_cost = loop_fn("test", train_set, testloader, model, criterion, optimizer, device)

Train:   0%|          | 0/187 [00:00<?, ?it/s]

IndexError: index out of range in self